In [1]:
# Import relevant libraries
import requests
import re
import json
import pandas as pd
import numpy as np

In [2]:
response = requests.get('https://www.smogon.com/dex/xy/pokemon/')

# pull JSON from response text via regex
data = "".join(re.findall(r'dexSettings = (\{.*\})', response.text))

# parse JSON
data = json.loads(data)

# query JSON to return relevant info
data = data.get('injectRpcs', [])[1][1].get('pokemon')

# make list of pokemon names
name_list = []
for row in data:
  name = row.get('name')

  # make names match original csv by removing hyphens and 
  x = name.split('-')
  if "Mega" in x:
    x.reverse()
    result = " ".join(x)
  else:
    result = " ".join(x)
  name_list.append(result)

# make list of tiers
tier_list = []
for row in data:
  tier_list.append(row.get('formats'))


In [3]:
# create df with name and tier values
tier_df = pd.DataFrame(name_list, columns = ["Name"])
tier_df.insert(1, "Tier", tier_list)
tier_df

,Name,Tier
0,Bulbasaur,[LC]
1,Ivysaur,[NFE]
2,Venusaur,[RU]
3,Charmander,[LC]
4,Charmeleon,[NFE]
...,...,...
876,Darmanitan Zen,[UU]
877,Mega Kangaskhan,[Uber]
878,Meloetta Pirouette,[RU]
879,Mega Aerodactyl,[UU]


In [4]:
tier_df['Tier'] = tier_df['Tier'].str[0]
tier_df

,Name,Tier
0,Bulbasaur,LC
1,Ivysaur,NFE
2,Venusaur,RU
3,Charmander,LC
4,Charmeleon,NFE
...,...,...
876,Darmanitan Zen,UU
877,Mega Kangaskhan,Uber
878,Meloetta Pirouette,RU
879,Mega Aerodactyl,UU


In [5]:
pokemon_df = pd.read_csv('Resources/pokemon.csv')
pokemon_df

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...
795,796,Diancie,Rock,Fairy,50,100,150,100,150,50,6,True
796,797,Mega Diancie,Rock,Fairy,50,160,110,160,110,110,6,True
797,798,Hoopa Confined,Psychic,Ghost,80,110,60,150,130,70,6,True
798,799,Hoopa Unbound,Psychic,Dark,80,160,60,170,130,80,6,True


In [6]:
# standardize pokenames and combine df's 
pokemon_df['Name'] = pokemon_df['Name'].str.replace('Forme', '')
pokemon_df['Name'] = pokemon_df['Name'].str.replace('Size', '')
pokemon_df['Name'] = pokemon_df['Name'].str.replace('Mode', '')
pokemon_df.set_index('#')


,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
#,,,,,,,,,,,
1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
5,Charmander,Fire,NaN,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...
796,Diancie,Rock,Fairy,50,100,150,100,150,50,6,True
797,Mega Diancie,Rock,Fairy,50,160,110,160,110,110,6,True
798,Hoopa Confined,Psychic,Ghost,80,110,60,150,130,70,6,True


In [7]:
pokemon_df = pokemon_df.merge(tier_df, on='Name')


In [10]:
pokemon_df = pokemon_df.set_index('#')

In [11]:
pokemon_df.sort_values('Name')

,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Tier
#,,,,,,,,,,,,
511,Abomasnow,Grass,Ice,90,92,75,92,85,60,4,False,NU
69,Abra,Psychic,NaN,25,20,15,105,55,90,1,False,LC
393,Absol,Dark,NaN,65,130,60,75,60,75,3,False,RU
679,Accelgor,Bug,NaN,80,70,40,100,60,145,5,False,RU
154,Aerodactyl,Rock,Flying,80,105,65,60,75,130,1,False,RU
...,...,...,...,...,...,...,...,...,...,...,...,...
287,Zigzagoon,Normal,NaN,38,30,41,30,41,60,3,False,LC
633,Zoroark,Dark,NaN,60,105,60,120,60,105,5,False,RUBL
632,Zorua,Dark,NaN,40,65,40,80,40,65,5,False,LC
